## About the analysis

The purpose of this analysis is to identify business categories in New York City that have seen dramatic changes in their numbers of business licenses, particularly since the onset of the COVID-19 pandemic. The central questions we'll be asking are: what kinds of businesses are thriving post-pandemic, and which ones are not? 

The data dictionary for the dataset can be found in this project's repository in the "data" folder. 

## Step 1: Imports

First, import the libraries that we'll need, as well as the data to be analyzed. For more information about the libraries and data needed for this project, refer to the read.me file in the project's repository. 

In [32]:
## import libraries
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option("display.max_columns", None)

In [33]:
## read excel sheet
licenses = pd.read_excel("../data/issued_licenses.xlsx")

## Step 2: find out basic information about the dataset

We should begin by learning more about the dataset, including what its columns are and how many rows it contains. 

But we also need to determine which column we want to use for our analysis. If we want to see how much businesses have grown, we'll need to ensure that we're counting businesses just once in our analysis (unless that business is a chain). Businesses can file for several licenses over their lifetimes, and some of them file for multiple licenses using the same unique ID. 

In [34]:
## what columns does the dataset have and how many? what kind of datatypes are in each of these columns?
licenses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59991 entries, 0 to 59990
Data columns (total 31 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   License Number         59991 non-null  object        
 1   Business Name          59990 non-null  object        
 2   DBA/Trade Name         9307 non-null   object        
 3   Business Unique ID     59991 non-null  object        
 4   Business Category      59991 non-null  object        
 5   License Type           59991 non-null  object        
 6   License Status         59991 non-null  object        
 7   Initial Issuance Date  59991 non-null  datetime64[ns]
 8   Expiration Date        59962 non-null  datetime64[ns]
 9   Details                6743 non-null   object        
 10  Contact Phone          43726 non-null  object        
 11  Address Type           45955 non-null  object        
 12  Building Number        45148 non-null  object        
 13  S

In [35]:
## how many rows does the dataset have? 
len(licenses)

59991

In [36]:
## which business names show up multiple times? 
licenses.groupby(by = ["Business Name"]).size().sort_values(ascending = False)

Business Name
T-MOBILE NORTHEAST LLC                 289
SP PLUS CORPORATION                    260
AT&T MOBILITY SUPPLY, LLC              121
ECOATM LLC                              99
OFFICE SUPERSTORE EAST LLC              88
                                      ... 
FRUIT TREE CORP.                         1
FRUITS AND VEGETABLES BROTHERS, INC      1
FRUTERA EL BUEN CAMINO CORP.             1
FRUTERIA SAN MIGUEL CORP                 1
ƒ∞smail Alata≈ü                          1
Length: 50719, dtype: int64

In [37]:
## how many unique business names are there? 
licenses["Business Name"].nunique()

50719

In [38]:
## which unique IDs show up multiple times? 
licenses["Business Unique ID"].value_counts()

Business Unique ID
BA-1305489-2022    10
BA-1302088-2022     7
BA-1123441-2022     6
BA-1103941-2022     6
BA-1278926-2022     5
                   ..
BA-1573837-2023     1
BA-1571642-2023     1
BA-1576112-2023     1
BA-1578809-2023     1
BA-1397082-2022     1
Name: count, Length: 54355, dtype: int64

In [39]:
## how many unique business IDs are there?

licenses["Business Unique ID"].nunique()

54355

## Step 3: Making additional changes to the dataset for analysis

Before we make any additional dataframes, let's separate the year from the date in which each business was first issued a license. This will help us analyze how business categories have fluctuated over time. 

In [40]:
## creating a new column in the df "licenses" that holds the year in which licenses are issued
licenses["year_issued"] = licenses["Initial Issuance Date"].dt.year

Based on what we did in step 2, it doesn't seem efficient to use only the business name column, or only the business unique id column, to conduct our analysis. Because we want to analyze business growth and decline, it may make more sense to tie our analysis to each business's location. 

In [41]:
## create a new column that creates a unique code for each biz based on its name and street address
licenses["identifier"] = licenses['Business Name'] + licenses['Street1'].astype(str)

In [42]:
## remove duplicates of businesses based on the unique identifiers we just made
no_dupes = licenses.drop_duplicates(subset=['identifier'], keep='last')
no_dupes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 53559 entries, 0 to 59990
Data columns (total 33 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   License Number         53559 non-null  object        
 1   Business Name          53558 non-null  object        
 2   DBA/Trade Name         7668 non-null   object        
 3   Business Unique ID     53559 non-null  object        
 4   Business Category      53559 non-null  object        
 5   License Type           53559 non-null  object        
 6   License Status         53559 non-null  object        
 7   Initial Issuance Date  53559 non-null  datetime64[ns]
 8   Expiration Date        53545 non-null  datetime64[ns]
 9   Details                6152 non-null   object        
 10  Contact Phone          37878 non-null  object        
 11  Address Type           39951 non-null  object        
 12  Building Number        39441 non-null  object        
 13  Street

In [43]:
## no more duplicates!
no_dupes["identifier"].value_counts()

identifier
HEALTHGUARD DME INC.BROAD ST                       1
CARUCCI, JOSEPHW 5TH ST                            1
ROBEL AUTO REPAIR INC.MESEROLE ST                  1
REVOLUTION RICKSHAWS L.L.C.10 AVENUE               1
ELISEO T ALVAREZ BARRENOVICTORY BLVD               1
                                                  ..
WILLIAMSBURG ORGANIC DELI 1 CORPUNION AVE          1
ANGELICA ARRIETAnan                                1
ARTURO D MARTINEZ RODRIGUEZnan                     1
Gabriel Arsenisnan                                 1
NEW YORK ENERGY CONSERVATION CO., INC.HARRIS RD    1
Name: count, Length: 53558, dtype: int64

While there are no longer any duplicates, we've ignored an important data point: license statuses. I'm choosing to ignore that for the moment, so that we can get a broad picture of business growth even if that business's license expired. 

## Step 4: Analysis

Now, we can moved forward with conducting our analysis. As a reminder, we want to discover:
- which businesses types are thriving post-COVID?
- which businesses are doing poorly post-COVID?

In [44]:
## how many businesses in each business category?
no_dupes["Business Category"].value_counts().to_frame()

,count
Business Category,
Home Improvement Contractor,16457
Tobacco Retail Dealer,4825
Secondhand Dealer - General,3671
Sightseeing Guide,3263
Locksmith,2673
Electronics Store,2613
Tow Truck Driver,2584
Garage & Parking Lot,2058
General Vendor,2017


In [45]:
## how many licenses were issued each year according to the business category?
no_dupes.groupby(["Business Category"])["year_issued"].value_counts().to_frame()

count
Business Category   year_issued       
Bingo Game Operator 1900            25
                    2023             3
                    2014             2
                    2015             1
                    2019             1
...                                ...
Tow Truck Driver    2000            19
                    1994            16
                    1998            16
                    2005            11
                    1900             1

[818 rows x 1 columns]

In [46]:
## reset the index and set the code above to a dataframe
placeholder = no_dupes.groupby(["Business Category"])["year_issued"].value_counts().to_frame()
changes = placeholder.reset_index()
changes

,Business Category,year_issued,count
0,Bingo Game Operator,1900,25
1,Bingo Game Operator,2023,3
2,Bingo Game Operator,2014,2
3,Bingo Game Operator,2015,1
4,Bingo Game Operator,2019,1
...,...,...,...
813,Tow Truck Driver,2000,19
814,Tow Truck Driver,1994,16
815,Tow Truck Driver,1998,16
816,Tow Truck Driver,2005,11


In [47]:
## info about the new df we just made 
changes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 818 entries, 0 to 817
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Business Category  818 non-null    object
 1   year_issued        818 non-null    int32 
 2   count              818 non-null    int64 
dtypes: int32(1), int64(1), object(1)
memory usage: 16.1+ KB


In [50]:
## create a pivot table 
output = changes.pivot_table(
    values="count",
    index=["year_issued"],
    columns=["Business Category"]
)

output

Business Category,Bingo Game Operator,Booting Company,Car Wash,Commercial Lessor - Bingo,Construction Labor Provider,Dealer In Products For The Disabled,Debt Collection Agency,Electronic & Home Appliance Service Dealer,Electronic Cigarette Dealer,Electronics Store,Employment Agency,Games of Chance - Bell Jar,Games of Chance - Las Vegas / Casino Nights,"Games of Chance - Raffle with Net Proceeds Over $30,000","Games of Chance - Raffle with Net Proceeds Under $30,000",Garage & Parking Lot,General Vendor,General Vendor Distributor,Home Improvement Contractor,Horse Drawn Cab Driver,Horse Drawn Cab Owner,Industrial Laundry,Industrial Laundry Delivery,Laundries,Locksmith,Locksmith Apprentice,Newsstand,Pawnbroker,Pedicab Business,Pedicab Driver,Process Server Individual,Process Serving Agency,Scale Dealer/Repairer,Scrap Metal Processor,Secondhand Dealer - Auto,Secondhand Dealer - General,Sightseeing Bus,Sightseeing Guide,Specialized Vending License - MFV,Stoop Line Stand,Storage Warehouse,Third Party Food Delivery Service,Ticket Seller Business,Ticket Seller Individual,Tobacco Retail Dealer,Tow Truck Company,Tow Truck Driver
year_issued,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1900,25.0,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,10.0,NaN,6.0,6.0,2.0,4.0,NaN,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,3.0,NaN,2.0,NaN,NaN,NaN,NaN,3.0,1.0,1.0
1994,NaN,NaN,NaN,NaN,NaN,NaN,9.0,13.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,19.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,NaN,NaN,NaN,1.0,NaN,79.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,16.0
1995,NaN,NaN,NaN,NaN,NaN,11.0,6.0,1.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,11.0,NaN,NaN,8.0,4.0,1.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,7.0,12.0,NaN,11.0,NaN,4.0,1.0,NaN,NaN,NaN,NaN,NaN,82.0
1996,NaN,NaN,NaN,NaN,NaN,2.0,1.0,15.0,NaN,12.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,26.0,34.0,3.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,1.0,NaN,1.0,2.0,2.0,2.0,NaN,110.0,NaN,18.0,NaN,NaN,NaN,NaN,NaN,NaN,45.0
1997,NaN,NaN,NaN,NaN,NaN,14.0,9.0,5.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,120.0,NaN,NaN,58.0,5.0,NaN,NaN,NaN,NaN,74.0,NaN,NaN,3.0,NaN,NaN,2.0,NaN,1.0,1.0,2.0,22.0,NaN,38.0,NaN,7.0,7.0,NaN,NaN,NaN,NaN,NaN,20.0
1998,NaN,NaN,NaN,NaN,NaN,6.0,5.0,6.0,NaN,5.0,1.0,NaN,NaN,NaN,NaN,16.0,NaN,NaN,79.0,7.0,1.0,NaN,NaN,NaN,50.0,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,2.0,9.0,24.0,NaN,22.0,NaN,11.0,NaN,NaN,NaN,NaN,NaN,NaN,16.0
1999,NaN,NaN,NaN,NaN,NaN,3.0,9.0,5.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,19.0,NaN,NaN,78.0,4.0,NaN,NaN,NaN,NaN,67.0,NaN,21.0,1.0,NaN,NaN,3.0,NaN,1.0,2.0,14.0,39.0,NaN,28.0,NaN,7.0,1.0,NaN,NaN,NaN,NaN,NaN,23.0
2000,NaN,NaN,NaN,NaN,NaN,4.0,6.0,4.0,NaN,12.0,8.0,NaN,NaN,NaN,NaN,18.0,NaN,NaN,64.0,4.0,1.0,NaN,NaN,NaN,31.0,NaN,15.0,1.0,NaN,NaN,2.0,NaN,NaN,NaN,5.0,20.0,NaN,42.0,NaN,10.0,NaN,NaN,NaN,NaN,215.0,NaN,19.0
2001,NaN,NaN,NaN,NaN,NaN,8.0,13.0,9.0,NaN,15.0,4.0,NaN,NaN,NaN,NaN,26.0,NaN,NaN,68.0,2.0,1.0,NaN,NaN,NaN,44.0,NaN,1.0,1.0,NaN,NaN,2.0,3.0,NaN,NaN,7.0,42.0,NaN,20.0,NaN,7.0,2.0,NaN,NaN,NaN,66.0,NaN,24.0


In [52]:
## write the pivot table to a csv file
output.to_csv("../output/output.csv", index = False)